In [ ]:
from __future__ import division
import networkx as nx
import matplotlib.pyplot as plt
import pylab
import scipy as sp
import numpy as np
import itertools
import copy
import math
import random

N =  50                  # Number of nodes
rw = 700                 # Wealth value of rich
pw = 300                # Wealth value of poor
we_arr = [rw, pw]       # Array storing both the wealth values
pr = 0.5                # Probability of being rich
prob_arr = [pr, 1-pr]  # Array storing probability of wealth allotment values
M = 20                  # Number of rounds
co = 50                 # Unitary Cost of co-operation
be = 100                # Unitary Benefit of co-operation
S = 50                  # Number of iterations
pl = 0.3                # Initial probability of a link being formed between any two nodes


def inv_logit(y):
    inv_logit_out =(np.exp(y)/(1+np.exp(y)))
    return inv_logit_out

def gini_calc(x):       #IN: Wealth values across the population, OUT: Gini Coefficient
    sum2=0
    for i in range(len(x)):
        sum1=0
        for j in range(len(x)):
            sum1=sum1+np.abs(x[i]-x[j])
        sum2=sum2+sum1;
    
    gini=sum2/(2*len(x)**2*np.mean(x));
    return gini

node_arr = np.arange(N)
re_ar = np.arange(0,1.1,0.1)

cc_deg_ar = np.zeros([len(re_ar),S])         # Array for storing average degree of cooperators in a cooperative environment
cd_deg_ar = np.zeros([len(re_ar),S])         # Array for storing average degree of cooperators in a selfish environment
dc_deg_ar = np.zeros([len(re_ar),S])         # Array for storing average degree of defectors in a cooperative environment
dd_deg_ar = np.zeros([len(re_ar),S])         # Array for storing average degree of defectors in a selfish environment
ce_deg_ar = np.zeros([len(re_ar),S])         # Array for storing average degree of cooperators in a NEUTRAL environment
de_deg_ar = np.zeros([len(re_ar),S])         # Array for storing average degree of defectors in a NEUTRAL environment

flag = 0

for h1 in range(len(re_ar)):
    
    re = re_ar[h1]
    for h in range(S):       
        # Flags or counts for total no. of different strategies in different environments
        cc = 0
        cd = 0
        dc = 0
        dd = 0
        ce = 0
        de = 0
        # Counts of degree for different strategies in different environments
        cc_deg = 0
        cd_deg = 0
        dc_deg = 0
        dd_deg = 0
        ce_deg = 0
        de_deg = 0
                
        G = nx.erdos_renyi_graph(N, pl)
        A = nx.adjacency_matrix(G)
        E = G.number_of_edges()
        No_of_pairs = (N*(N-1))/2
        No_of_rewiring = round(No_of_pairs*re)
        Nre = int(No_of_rewiring)        
        init_wealth_array = np.random.choice(we_arr,N,p=prob_arr) 
        GINI = gini_calc(init_wealth_array)       
            
        for k in range(M):                 # Continuing the game for M rounds
            
            if (k==0):                    # First round                
                decision_array = np.zeros(N)
                for i in range(N):         # For all nodes in the network
                    nn = G.degree(i)
                    a = np.random.random()                    
                    p1 = 0.7
                    if (a <= p1):
                        decision_array[i] = 1       # Node-i decides to be cooperative                    
                        init_wealth_array[i] = init_wealth_array[i] - co*nn   # Updatation of wealth of a cooperative node
                        for l in range(N):          # Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue

                    else:
                        decision_array[i] = 0          # Node-i decides to be non-cooperative
                        continue   
               
            else:                     # Decision making in subsequent rounds
                new_decision_array = np.zeros(N)
                
                for i in range(N):         # For all nodes in the network
                    nn = G.degree(i)                                    
                    cop = 0                     # Initializing the no of cooperator neighbours
                    for j in range(N):
                        if (A[i,j] == 1):               # If a link exists between nodes i and j
                            if (decision_array[j] == 1):        # If link j cooperated in last round
                                cop += 1
                            elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                                continue
                        else:
                            continue
                
                    sum_neighbor_wealth = 0
                    for j in range(N):			# Average wealth of the neighborhood
                        if (A[i,j] == 1):
                            sum_neighbor_wealth += init_wealth_array[j]
                        else:
                            continue
                 
                    if (nn>0):
                        av_neighbor_wealth = (sum_neighbor_wealth)/nn
                        diff_wealth = av_neighbor_wealth - (init_wealth_array[i])
                        pc = p1 + (1-p1)*(np.tanh(0.001*diff_wealth)) - 0.10 
                    else:
                        pc = p1                   
                
                
                    if (nn>0):              # For postive number of nearest neighbours
                        cop_rat = float(cop/nn)                    
                        if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5
                            
                            c = np.random.random()
                            if (c < pc):
                                new_decision_array[i] = 1       # Node-i decides to be cooperative                          
                            else:
                                new_decision_array[i] = 0

                        elif (cop_rat < 0.5):          # Cooperation parameter is lesser than 0.5
                            
                            c = np.random.random()
                            if (c < pc):                        
                                new_decision_array[i] = 0
                            else:
                                new_decision_array[i] = 1
                            
                        elif (cop_rat == 0.5):     # Neutral environment
                              
                              c = np.random.random()
                              if (c<=pc):
                                  new_decision_array[i] = 1
                              else:
                                  new_decision_array[i] = 0
                                    
                    else:
                        new_decision_array[i] = 0
                        
                for i in range(N):
                    if (new_decision_array[i] == 1): 
                        init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))   # Updatation of wealth of a cooperative node
                        for l in range(N):          	# Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue
                    else:
                        continue       
                
                
                decision_array = copy.copy(new_decision_array)         
        
            
            # Rewiring step
                 
            temp_array = []                           
            
                                                                          
            while (len(temp_array) < Nre) :            
                i = random.choice(node_arr)
                j = random.choice(node_arr)
                    
                if (i==j):
                    continue
                else:
                
                    if (i,j) in temp_array:
                        continue
                    elif (j,i) in temp_array:
                        continue
                    else:
                        temp_array.append((i,j))
                        temp = [i,j]
                                                             
                        if (A[i,j] == 1):                              # If a link exists between the selected pair
                            b = random.choice(temp)
                            
                            if (b == i):                               # If node-i is given the choice to break or not
                                if (decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 0.7):
                                        G.remove_edge(i,j)                 # Breakage of link for node-i disagreeing
                                  
                                    else:
                                        continue                        # Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 0.87):
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                            else:                                   # If node-j is given the choice to break or not	
                                if (decision_array[i] == 0):
                                    c = np.random.random()
                                    if (c < 0.7):                                    
                                        G.remove_edge(i,j)         # Breakage of link for node-j disagreeing
                                    else:
                                        continue			# Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 0.87):
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                                                                
            
                        else:                 # If a link does not exist between the selected pair
                            if (decision_array[i] == 1 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 0.93):
                                    G.add_edge(i,j)
                                else:
                                    continue
                            
                            if (decision_array[i] == 0 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 0.80):
                                    continue
                                else:
                                    G.add_edge(i,j)
                        
                            if (decision_array[i] == 0 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 0.70):
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
                            if (decision_array[i] == 1 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 0.70):
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
            A = nx.adjacency_matrix(G)   # Adjacency matrix of the network
            
                        
            env_array = np.zeros(N)
            for i in range(N):         # For all nodes in the network                                                   
                cop = 0                     # Initializing the no of cooperator neighbours
                for j in range(N):
                    if (A[i,j] == 1):               # If a link exists between nodes i and j
                        if (decision_array[j] == 1):        # If link j cooperated in last round
                            cop += 1
                        elif (decision_array[j] == 0):      # If link j did not cooperate in last round
                            continue
                    else:
                        continue
                if (G.degree(i)>0):
                    cop_rat = float(cop/G.degree(i))                    
                    if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5
                        env_array[i] = 1
                        if (decision_array[i] == 1):
                            cc += 1
                        else:
                            dc += 1
                    elif (cop_rat == 0.5):    # Neutral environment
                        env_array[i] = 2
                        if (decision_array[i] == 1):
                            ce += 1
                        else:
                            de += 1 
                    elif(cop_rat < 0.5):       # Selfish environment
                        env_array[i] = 0
                        if (decision_array[i] == 1):
                            cd += 1
                        else:
                            dd += 1
                else:
                    env_array[i] = 0
                    if (decision_array[i] == 1):
                        cd += 1
                    else:
                        dd += 1                        
                   
            cc_idx = [j for j in range(len(env_array)) if env_array[j] == 1 and decision_array[j] == 1]
            dc_idx = [j for j in range(len(env_array)) if env_array[j] == 1 and decision_array[j] == 0]
            cd_idx = [j for j in range(len(env_array)) if env_array[j] == 0 and decision_array[j] == 1]
            dd_idx = [j for j in range(len(env_array)) if env_array[j] == 0 and decision_array[j] == 0]
            ce_idx = [j for j in range(len(env_array)) if env_array[j] == 2 and decision_array[j] == 1]
            de_idx = [j for j in range(len(env_array)) if env_array[j] == 2 and decision_array[j] == 0]
            
                            
            for i in cc_idx:
                cc_deg += G.degree(i)
           
            for i in dc_idx:
                dc_deg += G.degree(i)
            
            for i in cd_idx:
                cd_deg += G.degree(i)
           
            for i in dd_idx:
                dd_deg += G.degree(i)
            
            for i in ce_idx:
                ce_deg += G.degree(i)
           
            for i in de_idx:
                de_deg += G.degree(i)          
                
        # Normalizing the total degree for each type by total no. of each types   
        
        if (cc>0):
            cc_deg_ar[h1,h] = cc_deg/cc
        
        if (dc>0):
            dc_deg_ar[h1,h] = dc_deg/dc
            
        if (cd>0):
            cd_deg_ar[h1,h] = cd_deg/cd
        
        if (dd>0):
            dd_deg_ar[h1,h] = dd_deg/dd
        
        if (ce>0):
            ce_deg_ar[h1,h] = ce_deg/ce
        
        if (de>0):
            de_deg_ar[h1,h] = de_deg/de
        
        
        flag += 1
        yo = (flag*100)/(len(re_ar)*S)
        print('Job completion percentage: %f %f %f' %(yo,h,h1))
        #print('Success!')
        
np.savez('degree_vs_re,p0=0.7_new', cc_deg_ar=cc_deg_ar,cd_deg_ar=cd_deg_ar,dc_deg_ar=dc_deg_ar,dd_deg_ar=dd_deg_ar,ce_deg_ar=ce_deg_ar,de_deg_ar=de_deg_ar)